In [16]:
#import required libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import regex as re
import time
from geopy.exc import GeocoderTimedOut 
from geopy.geocoders import Nominatim

In [17]:
#create empty lists to store the values
property_label=[]
property_price = []
description=[]
property_location = []
date_listed = []
link = []

#read the number of pages
for page in range(1,5):
    html_string = requests.get("https://www.gumtree.com.au/s-property-for-rent/melbourne/page-"+str(page)+"/c18364l3001317").content
    soup = BeautifulSoup(html_string, 'html.parser')
    
    
    content = soup.find_all("div",attrs={"class": "user-ad-collection-new-design"})
    for ad_list in content:
        for ad in ad_list.find_all('a'):
            ad_link = ad['href']
            link.append(ad_link)
            ad_basics = ad['aria-label'].split('\n')
            ad_title , price , location_date = ad_basics[0],ad_basics[1].strip(),ad_basics[2].strip()
            cost = re.findall('Price:([^.]+)',price)[0]       #extracting price
            property_price.append(cost)
            property_label.append(ad_title)
            loc_date=location_date.split('.')
            loc,date= loc_date[0],loc_date[1]        
            location= re.findall('Location:([^.]+)',loc)[0]     #extracting location 
            property_location.append(location)
            desc= ad.find('p',class_='user-ad-row-new-design__description-text').text
            description.append(desc)



In [18]:
#creating a dataframe
column = {'Title':property_label,'property description' : description, 'Price' : property_price, 'Location' :property_location,'Ad Link':link}
properties = pd.DataFrame(column)
properties

,Title,property description,Price,Location,Ad Link
0,Beautiful 3 bedroom unit available 8th of Jan ...,Modern three bedroom unit. Master bedroom is a...,"$1,955","Kingston Area, Clayton South",/s-ad/clayton-south/property-for-rent/beautifu...
1,Fully Furnished apartments - minimum one month...,We have apartments in Australia 108 for rent $...,$500,"Melbourne City, Southbank",/s-ad/southbank/property-for-rent/fully-furnis...
2,RESERVOIR 500 m from NORTHLAND BRAND NEW.,Brand New be the first person to occupy NOW co...,$440,"Darebin Area, Reservoir",/s-ad/reservoir/property-for-rent/reservoir-50...
3,FOR LEASE Large 2 Bedroom Fully Furnished Hou...,For Lease large 2 bedroom Fully Furnished hous...,$600 negotiable,"Maribyrnong Area, Yarraville",/s-ad/yarraville/property-for-rent/for-lease-l...
4,Furnished apartment for lease near RMIT.,Furnished near new apartment flexible lease te...,$450,"Melbourne City, Melbourne CBD",/s-ad/melbourne-cbd/property-for-rent/furnishe...
...,...,...,...,...,...
115,New modern two bedroom apartment 20mins to CBD...,Modern new two bedroom apartment 20 mins to Me...,$390,"Hume Area, Sunbury",/s-ad/sunbury/property-for-rent/new-modern-two...
116,Noble park north granny flat up for rent.,A granny flat dwelling on the same property wi...,"$2,000 negotiable","Greater Dandenong, Noble Park North",/s-ad/noble-park-north/property-for-rent/noble...
117,house for rent.,"looking for house for my self, can pay up to 2...",$250 negotiable,"Moreland Area, Brunswick West",/s-ad/brunswick-west/property-for-rent/house-f...
118,Renovated Malvern East Apartment Rental.,Where location matters! This renovated modern ...,$460,"Stonnington Area, Malvern East",/s-ad/malvern-east/property-for-rent/renovated...


In [19]:
#finding latitude and longitude of the property location using geopy
geolocator = Nominatim(user_agent='New_application')
for index, row in properties.iterrows():
    location = geolocator.geocode(row['Location'],timeout=10)
    if location == None:
        properties.loc[index,'latitude'] = 'Not available'
        properties.loc[index,'longitude'] = 'Not available'
    else:   
        lat,lng = location.latitude , location.longitude
        properties.loc[index,'latitude'] = lat
        properties.loc[index,'longitude'] = lng   
    

In [20]:
properties

,Title,property description,Price,Location,Ad Link,latitude,longitude
0,Beautiful 3 bedroom unit available 8th of Jan ...,Modern three bedroom unit. Master bedroom is a...,"$1,955","Kingston Area, Clayton South",/s-ad/clayton-south/property-for-rent/beautifu...,-37.9431,145.125
1,Fully Furnished apartments - minimum one month...,We have apartments in Australia 108 for rent $...,$500,"Melbourne City, Southbank",/s-ad/southbank/property-for-rent/fully-furnis...,-37.8254,144.964
2,RESERVOIR 500 m from NORTHLAND BRAND NEW.,Brand New be the first person to occupy NOW co...,$440,"Darebin Area, Reservoir",/s-ad/reservoir/property-for-rent/reservoir-50...,-37.7168,145.007
3,FOR LEASE Large 2 Bedroom Fully Furnished Hou...,For Lease large 2 bedroom Fully Furnished hous...,$600 negotiable,"Maribyrnong Area, Yarraville",/s-ad/yarraville/property-for-rent/for-lease-l...,-37.8159,144.89
4,Furnished apartment for lease near RMIT.,Furnished near new apartment flexible lease te...,$450,"Melbourne City, Melbourne CBD",/s-ad/melbourne-cbd/property-for-rent/furnishe...,-37.8153,144.962
...,...,...,...,...,...,...,...
115,New modern two bedroom apartment 20mins to CBD...,Modern new two bedroom apartment 20 mins to Me...,$390,"Hume Area, Sunbury",/s-ad/sunbury/property-for-rent/new-modern-two...,-37.5797,144.73
116,Noble park north granny flat up for rent.,A granny flat dwelling on the same property wi...,"$2,000 negotiable","Greater Dandenong, Noble Park North",/s-ad/noble-park-north/property-for-rent/noble...,-37.9513,145.191
117,house for rent.,"looking for house for my self, can pay up to 2...",$250 negotiable,"Moreland Area, Brunswick West",/s-ad/brunswick-west/property-for-rent/house-f...,-37.7633,144.943
118,Renovated Malvern East Apartment Rental.,Where location matters! This renovated modern ...,$460,"Stonnington Area, Malvern East",/s-ad/malvern-east/property-for-rent/renovated...,-37.8768,145.066


In [21]:
#plotting the map of melbourne using folium
import folium
mel_map = folium.Map([-37.840935, 144.946457], zoom_start=10)
mel_map

In [22]:
#plotting the details of property 
for index, row in properties.iterrows():
    if row['latitude'] != 'Not available':
        link_for_property = "https://www.gumtree.com.au"+row['Ad Link']
        folium.Marker([row['latitude'], row['longitude']],
                      popup=row['Price']+"\n"+'<a href='+link_for_property+' target="_blank">Property Link</a>',
                      tooltip=row['Location'],              
                     ).add_to(mel_map)        

In [23]:
mel_map